In [ ]:
from datascience import *
import numpy as np

import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
%matplotlib inline

## Midterms and Section Scores

In [ ]:
scores = Table.read_table('scores_by_section.csv')
scores

In [ ]:
# Look at the data by section
# How many students are in each section
scores.group('Section')
# What was the average score in each section?


In [ ]:
scores.group('Section',np.average).show(14)

### Possible Hypotheses

- **Null**: The Section 3 average is like the average of the *same number* of random scores from the class.
- **Alternative**: No, the Section 3 average is too low.

In [ ]:
# observed statistic
obs_av = 13.6667
# #students in section 3
students = 27


In [ ]:
np.average(scores.sample(students, with_replacement=False).column('Midterm'))

In [ ]:
# Sample from the scores repetitions times and save the average scores in averages
averages = make_array()
repetitions = 50000
for i in np.arange(repetitions):
    averages = np.append(averages, np.average(scores.sample(students, with_replacement=False).column('Midterm')))


In [ ]:
np.average(averages)
# Plot the Random sample average with 25 bins
Table().with_column('Sample Average',averages).hist(bins = 25)
# Add a point with the observed statistic (average from Section 3)
plots.scatter(obs_av, 0, s=30, color='red')
plots.plot([obs_av,obs_av], [0, .35], color ='red')
plots.plot([13.6,13.6], [0, .35], color ='yellow')


In [ ]:
# What proportion of the scores from the sample are below the observed average from Section 3
np.count_nonzero(averages < obs_av)/repetitions

In [ ]:
# How sensitive is this proportion to small perturbations in the observed average?
np.count_nonzero(averages < 13.6)/repetitions

## Smoking & Baby Birthweight

In [ ]:
baby = Table.read_table('baby.csv')
baby

In [ ]:
smoking_and_birthweight = baby.select('Maternal Smoker', 'Birth Weight')

In [ ]:
smoking_and_birthweight.group('Maternal Smoker')

In [ ]:
smoking_and_birthweight.hist('Birth Weight', group='Maternal Smoker')

In [ ]:
means = smoking_and_birthweight.group('Maternal Smoker', np.mean)
mw = means.column(1)

In [ ]:
mw.item(0) - mw.item(1)

## Deflategate

From the [Wells Report](https://www.washingtonpost.com/apps/g/page/sports/deflategate-the-full-ted-wells-report/1687/):

> On January 18, 2015, the New England Patriots and Indianapolis Colts played in the AFC Championship Game at Gillette Stadium in Foxborough, Massachusetts to determine which team would advance to Super Bowl XLIX. During the first half of the game, a question was raised by the Colts concerning the inflation level of the footballs being used by the Patriots. As a result, at halftime, members of the officiating crew assigned to the game, overseen by a senior officiating supervisor from the National Football League (the “NFL” or the “League”), tested the air pressure of footballs being used by each of the Patriots and the Colts. All eleven of the Patriots game balls tested measured below the minimum pressure level of 12.5 pounds per square inch (“psi”) allowed by Rule 2 of the Official Playing Rules of the National Football League (the “Playing Rules”) on both of two air pressure gauges used to test the balls. The four Colts balls tested each measured within the 12.5 to 13.5 psi range permitted under the Playing Rules on at least one of the gauges used for the tests

In [ ]:
football = Table.read_table('deflategate.csv')
football.show()

In [ ]:
football = football.drop(1, 2).with_column(
    'Combined', (football.column(1)+football.column(2))/2
    )
football.show()

## Causality
Consider the the effect of using Botulinum Toxin A as a treatment for lower back pain. Thirty one patients with low-back pain were randomized into treatment and control groups, with 15 given Botulinum Toxin A as *treatment* and 16 in given Saline as *control*. 

In [ ]:
bta = Table.read_table('bta.csv')
bta.show()